In [4]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

from ban_carbon_research.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

geo_pps = geopandas.read_file(PROCESSED_DATA_DIR / "Power_Plants")
rggi_pps = pd.read_csv(PROCESSED_DATA_DIR / "rggi-emissions-annual-facility.csv")

pps = geo_pps.merge(
    rggi_pps[rggi_pps["Year"] == 2024],
    left_on = "Plant_Code",
    right_on = "ORIS Code",
    how = "right",
    indicator = True
    )
# There are seven unmerged power plants. They were decomissioned in 2024.
#pps["_merge"].value_counts()
#unmerged = pps[pps["_merge"] == "right_only"]
pps = pps[pps["_merge"] == "both"].drop("_merge", axis = 1)
pps["County"].value_counts().sort_index()

County
Albany          2
Anne Arundel    2
Barnstable      1
Bergen          1
Berkshire       1
               ..
Washington      1
Windham         1
Worcester       3
Wyoming         1
Yates           1
Name: count, Length: 69, dtype: int64

In [ ]:
# Question: How much of the emissions are due to NYC?
ny_met_counties = [
    "Kings",
    "Queens",
    "New York",
    "Bronx",
    "Richmond",
    "Westchester",
    "Bergen",
    "Hudson",
    "Passaic",
    "Rockland",
    "Putnam",
    "Suffolk",
    "Nassau",
    "Middlesex",
    "Monmouth",
    "Ocean",
    "Somerset",
    "Essex",
    "Union",
    "Morris",
    "Sussex",
    "Hunterdon",
]
co2 = "CO2 Mass (Tons)"
print(pps.loc[pps["County"].isin(ny_met_counties), co2].sum() / pps[co2].sum())
print(pps.loc[pps["County"].isin(ny_met_counties), co2].sum())
# Answer: 41%, or ~30M metric tons CO2. Like Senegal, Zimbabwe, Guinea, half of Congo. More than Croatia

0.4082860563484762
33113311.803000003


In [35]:
# How much have power plants polluted up to October 7?
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ban_carbon_research.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

rggi = pd.read_csv(RAW_DATA_DIR / "rggi-emissions-quarterly-unit.csv")

rggi = rggi.loc[rggi["Year"].isin([2024, 2025]), ["Year", "Qtr", "CO2 Mass (Tons)"]]
rggi["t"] = rggi["Year"] + rggi["Qtr"]/10
rggi["co2"] = rggi["CO2 Mass (Tons)"]
rggi["co2"] = rggi["co2"].replace("No Data", np.nan)
rggi["co2"] = rggi["co2"].str.replace(",", "").astype(np.float64)

yearly = rggi[["Year", "co2"]].groupby("Year").sum()
quarterly = rggi[["Year", "Qtr", "co2"]].groupby(["Year", "Qtr"]).sum()

co2_2025 = quarterly.loc[(2025, 1), "co2"] + quarterly.loc[(2025, 2), "co2"]
co2_2024 = yearly.loc[2024, "co2"]
# Oct 7 is the 280th day of the year.
print("Answer: " + str(co2_2025 + (co2_2024-co2_2025) * (280-180) / 180))


# And how many tons will they emit on average every second?
tco2_avg = (co2_2024-co2_2025)/(180*24*3600)
print("TCO2/sec average: " + str(tco2_avg))
print(1/tco2_avg)

Answer: 63250711.91177777
TCO2/sec average: 2.871750206918724
0.34821970155717724


In [37]:
co2_2024 - co2_2025

np.float64(44661459.217999995)